In [8]:
search_for = 82
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002431154251098633
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 89788555
type: [1, 1, 1, 1, 82] 82
cases of this type: 45212176
10000 13.775533952666223
20000 14.093680846750605
30000 53.86311856887075
40000 47.14495096053789
50000 53.92916104222331
60000 60.06820337999605
70000 55.41537069437087
80000 59.04596109430025
90000 53.69015202008337
100000 44.874295066783425
110000 58.06085614950161
120000 53.95108082831028
130000 53.79951311551442
140000 52.980353413692455
150000 52.53362385559529
160000 47.203213714194696
170000 48.963785025805436
180000 45.26040862586882
190000 45.441419802953305
200000 44.50563961647398
210000 47.03698430182669
220000 45.77538318769108
230000 47.89534104069101
240000 45.01797396460732
250000 45.044945106676515
260000 51.20506831215845
270000 20.207614671990143
280000 8.000473767774105
290000 7.94632415129294
300000 26.602116356300645
310000 52.12531347287953
3

3090000 38.05314075310777
3100000 39.42166299481441
3110000 36.69907315882501
3120000 36.29664243226131
3130000 35.986715699646574
3140000 35.29556901889033
3150000 38.44199985112717
3160000 36.81980240268141
3170000 41.04951188687848
3180000 36.8239602519687
3190000 39.54997763387392
3200000 38.56557838890411
3210000 37.302336808421316
3220000 36.015592622970146
3230000 35.95478668211844
3240000 39.440369689006374
3250000 38.53971374998272
3260000 42.517367802044156
3270000 19.624261724398135
3280000 16.781125875816144
3290000 15.89694686634057
3300000 33.51363069173383
3310000 44.50424740659028
3320000 41.991421053313374
3330000 40.553209732803765
3340000 30.85301722389728
3350000 14.15458067424029
3360000 13.70370567013287
3370000 16.845848545940818
3380000 42.82155547263268
3390000 38.68596168036391
3400000 38.8528314878424
3410000 36.994192309199704
3420000 36.07586050709055
3430000 37.71637552362174
3440000 37.34488979409797
3450000 38.769478086525304
3460000 37.097854287389424
3

6220000 38.148690076306465
6230000 35.5019228868413
6240000 39.21992618250234
6250000 39.3737651856784
6260000 34.54208545200735
6270000 34.424433510270184
6280000 31.839005184410908
6290000 33.330034810543424
6300000 32.895181858951446
6310000 32.1051371537441
6320000 32.27820406681889
6330000 32.475528554981885
6340000 6.691780765660901
6350000 7.008322335771422
6360000 7.070682472976545
6370000 38.974128496429024
6380000 31.7293099518715
6390000 32.407862554634214
6400000 32.93621447064221
6410000 32.331910761676134
6420000 29.726023800974286
6430000 36.71476815805316
6440000 35.22587735261043
6450000 38.11863151429494
6460000 33.77289623555303
6470000 34.00601056636132
6480000 36.723080119504665
6490000 37.95072540004193
6500000 38.305436970458196
6510000 39.081732810172376
6520000 35.986698092647394
6530000 36.836365867296095
6540000 15.526871032737661
6550000 17.06937426370571
6560000 16.699145631721983
6570000 38.29631635907583
6580000 37.047728056639436
6590000 35.2743396683743

9330000 35.299648558775814
9340000 33.11466904581935
9350000 33.16892756741454
9360000 34.27896220742468
9370000 35.866522130996884
9380000 37.555968212021156
9390000 35.040706329993775
9400000 34.25674984630415
9410000 32.274501871015595
9420000 31.930774865776133
9430000 37.7377314871043
9440000 33.25406160720166
9450000 34.15920593952454
9460000 28.84745379498988
9470000 39.261149281557564
9480000 22.763043531903755
9490000 13.243603389825623
9500000 12.94740646555106
9510000 19.22921056920737
9520000 34.57357061879317
9530000 31.970944919382564
9540000 31.510792638500465
9550000 34.14764298469454
9560000 29.792178674608603
9570000 35.4511344459893
9580000 30.680938229596318
9590000 30.792075985458126
9600000 26.741259711657197
9610000 32.057145703713495
9620000 29.1910400776576
9630000 34.16548401416437
9640000 20.64828982217699
9650000 5.66489754501813
9660000 5.964871187882457
9670000 8.824275087514717
9680000 34.84795481950419
9690000 32.2744609910807
9700000 27.3070642003867
97

12350000 30.35434624963277
12360000 30.13072351876779
12370000 27.79678985683908
12380000 30.646641949843993
12390000 31.88845535359999
12400000 14.060410138169726
12410000 6.79588384329776
12420000 6.481671397391524
12430000 17.617210268570485
12440000 34.510590869094166
12450000 27.798627743144333
12460000 31.96998100935489
12470000 29.55212586664061
12480000 30.874899478272127
12490000 30.775241238514084
12500000 29.41847601189633
12510000 30.602357514253754
12520000 29.90904783063024
12530000 15.310602068968747
12540000 15.400826477573844
12550000 15.825476551641326
12560000 28.472569465730256
12570000 30.112436973978877
12580000 28.790166319960257
12590000 29.939976359041797
12600000 30.883264517545996
12610000 29.70089969071001
12620000 31.728564211621084
12630000 33.61452288577613
12640000 30.037042163716354
12650000 31.229431560635998
12660000 34.378173491910964
12670000 34.64188583794206
12680000 37.660239232838315
12690000 35.60365819076117
12700000 32.089995187385085
1271000

...

78190000 2.99964676011314
78200000 2.9047736377708118
78210000 3.026509411399563
78220000 3.123913776175082
78230000 1.8245638431798088
78240000 1.61657481613285
78250000 3.637046654880643
78260000 2.982671802925236
78270000 3.060219123578601
78280000 3.135247849976619
78290000 3.2363055438921187
78300000 3.1485885080594485
78310000 3.1036424781608583
78320000 2.993717176577681
78330000 3.0709117123598855
78340000 3.1469647558813296
78350000 2.264042261552115
78360000 1.9781934215483732
78370000 2.231184659556846
78380000 3.0338779032219447
78390000 2.918450658187171
78400000 2.894095862497091
78410000 3.043097868593699
78420000 3.176903686440951
78430000 1.3605580039569736
78440000 1.0383845619628165
78450000 1.3584589651457801
78460000 2.888069773821831
78470000 2.917862194603317
78480000 2.769717976352175
78490000 2.9848101840251684
78500000 2.9921851438421343
78510000 2.2471660656796892
78520000 1.5979001995268465
78530000 3.0462311783037253
78540000 2.9803601161938573
78550000 3.0

81090000 0.5848429061422082
best so far: 0
type: [1, 1, 82, 1, 1] 82
cases of this type: 6724
best so far: 0
type: [1, 2, 1, 1, 41] 82
cases of this type: 5651522
81100000 0.5545003389832377
81110000 1.6027216693304314
81120000 2.6239716818127703
81130000 2.401835248608788
81140000 2.7105176531603603
81150000 2.4620694121741256
81160000 2.4214241817240914
81170000 0.7529213993468549
81180000 1.7659208422427044
81190000 2.4484993941906095
81200000 2.546718383612732
81210000 2.6304921633643246
81220000 2.232452811600069
81230000 2.0829844334022205
81240000 1.4915258432467116
81250000 1.1416694378434618
81260000 2.2447496636669175
81270000 2.3103880608288114
81280000 2.2144078351154923
81290000 2.3651054796424176
81300000 1.867553289317224
81310000 0.724429918081661
81320000 1.865962869100173
81330000 2.1432485575740206
81340000 2.228819103733698
81350000 2.285035014828112
81360000 2.3200460428908136
81370000 1.2132437695673108
81380000 1.3049520311443343
81390000 1.35936048633188
8140000

83990000 0.32599342755513055
84000000 0.4544501353326771
84010000 0.4479057822945714
84020000 0.40412778019958073
84030000 0.4216332051980827
84040000 0.42144707429269945
84050000 0.41463907570554154
84060000 0.4010892549362116
84070000 0.3242758763355017
84080000 0.3913825624359979
84090000 0.4106273187457191
84100000 0.4584340343037247
84110000 0.4322546100439296
84120000 0.36958399153669674
84130000 0.3688462961388628
84140000 0.3585406418620547
84150000 0.36171339072148
84160000 0.3822987022724748
84170000 0.4462140488848752
84180000 0.43323088279936045
84190000 0.43587574553151925
84200000 0.2835889867464039
84210000 0.3996219002001815
84220000 0.3179740310301383
84230000 0.4068997767825259
84240000 0.39310208469321334
84250000 0.43194777120858435
84260000 0.4275629936555029
84270000 0.595404847829938
84280000 1.7005538152189055
84290000 1.4627903841533265
84300000 1.5604273167652885
84310000 1.646429949950278
84320000 1.549363079671429
84330000 0.8586304957598117
84340000 0.42212

86890000 0.17374798776159683
best so far: 0
type: [1, 41, 1, 1, 2] 82
cases of this type: 656
best so far: 0
type: [1, 41, 1, 2, 1] 82
cases of this type: 328
best so far: 0
type: [1, 41, 2, 1, 1] 82
cases of this type: 164
best so far: 0
type: [1, 82, 1, 1, 1] 82
cases of this type: 82
best so far: 0
type: [2, 1, 1, 1, 41] 82
cases of this type: 2825761
86900000 0.4279008386374844
86910000 0.6860034242934983
86920000 0.7364830709232886
86930000 0.3580931606342726
86940000 0.7801783493538367
86950000 0.8604534071874619
86960000 0.7182666954362393
86970000 0.41364751083532975
86980000 0.7408410037996372
86990000 0.6857436259696881
87000000 0.26400930117603805
87010000 0.667705038536191
87020000 0.6631273008486629
87030000 0.5808861111139588
87040000 0.4718146015337109
87050000 0.6130629308051533
87060000 0.6666138364164698
87070000 0.29772088917126255
87080000 0.6579894859643115
87090000 0.6404766162603431
87100000 0.4806209382941326
87110000 0.4466438324366344
87120000 0.70089238236708

89640000 0.038960043141345185
89650000 0.023816269488334655
89660000 0.030680096548034084
89670000 0.029385315676695768
89680000 0.028031626079976557
89690000 0.012264108596411015
89700000 0.02224562480211258
89710000 0.019974562019407748
best so far: 0
type: [2, 1, 1, 41, 1] 82
cases of this type: 68921
89720000 0.012367804671691525
89730000 0.003860467296573851
89740000 0.00324458589553833
89750000 0.002373918496900135
89760000 0.0018918073213431569
89770000 0.0011383620528380076
89780000 0.0004580922469496727
best so far: 0
type: [2, 1, 41, 1, 1] 82
cases of this type: 1681
best so far: 0
type: [2, 41, 1, 1, 1] 82
cases of this type: 41
best so far: 0
type: [41, 1, 1, 1, 2] 82
cases of this type: 16
best so far: 0
type: [41, 1, 1, 2, 1] 82
cases of this type: 8
best so far: 0
type: [41, 1, 2, 1, 1] 82
cases of this type: 4
best so far: 0
type: [41, 2, 1, 1, 1] 82
cases of this type: 2
best so far: 0
type: [82, 1, 1, 1, 1] 82
cases of this type: 1
best so far: 0
all cases: 89788555
b